<div>
    <h1><center style="background-color:#8394bf; color:white;">Among Us</center></h1>
</div>

<div>
 <img src="https://i.imgur.com/6MgzTEM.gif">

<br>
Among Us, a 2018 game, gained a lot of popularity during the summer of 2020. 
This dataset contains game stats of different users. 
<br>
 <br>
</div>

> 📖 **Dataset columns description**:
* `Game Completed Date` : Game completion date
* `Team` : Team of the player
* `Outcome` : Result of the game
* `Task completed` : Number of tasks completed by a crewmate
* `All taks completed` : Whether all the tasks were completed by the crewmates during the game
* `Murdered` : Whether a crewmate was murdered or not
* `Imposter Kills` : Number of crewmates killed by imposters
* `Game length` : Duration of the game
* `Ejected` : Whether a player was removed during the game
* `Sabotages fixed` : Number of sabotages fixed by by the crewmates
* `Time to complete all tasks` : Time taken by crewmates to complete all the tasks
* `Rank Change` : After playing 3 games, players are assigned a competitive rank that is affected by every game played
* `Region/Game Code` : Server Region and game code of a match

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div>  
<h3><center style="background-color:#8394bf; color:white;"><strong>Importing Libraries 📚</strong></center></h3>
</div>

In [ ]:
!pip install bar_chart_race

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px 
import glob
import bar_chart_race as bcr

<div>  
<h3><center style="background-color:#8394bf; color:white;"><strong>Custom color palette🎨</strong></center></h3>
</div>

In [ ]:
custom_colors = ["#6b2fbb","#132ed1","#3f47fe","#d6e0f0","#117f2d","#50ef39","#38fedc","#71491e","#c51111","#ed54ba","#ef7d0d","#f5f557"]
customPalette = sns.set_palette(sns.color_palette(custom_colors))

In [ ]:
sns.palplot(sns.color_palette(custom_colors),size=0.8)
plt.tick_params(axis='both', labelsize=0, length = 0)

In [ ]:
path = '../input/among-us-dataset' 
all_files = glob.glob(path + "/*.csv")

In [ ]:
li = []
idx = 0
usr = 0

for filename in all_files:
    usr+=1
    df = pd.read_csv(filename, index_col=None, header=0)
    df['User ID']=usr
    li.append(df)

In [ ]:
li[9].head()

<div>  
<h3><center style="background-color:#8394bf; color:white;"><strong>Merging all the files</strong></center></h3>
</div>

In [ ]:
df_merged = pd.concat(li)
df_merged = df_merged.reset_index(drop=True)

In [ ]:
df_merged[['Region','Game Code']] = df_merged['Region/Game Code'].str.split("/",expand=True) 
df_merged = df_merged.drop(columns=['Region/Game Code'])

In [ ]:
df_merged.head()

<div>  
<h3><center style="background-color:#8394bf; color:white;"><strong>Modifying existing dataset columns</strong></center></h3>
</div>

In [ ]:
timezones=[]
for index, row in df_merged.iterrows():
    tz = df_merged['Game Completed Date'][index][-3:]
    if tz not in timezones:
        timezones.append(tz)

In [ ]:
timezones

> 📌 Note: Since there is just one time zone in our dataset, we can safely drop this value.

In [ ]:
date_val = []
time_val = []
for index, row in df_merged.iterrows():
    st = df_merged['Game Completed Date'][index].split()
    date=st[0]
    t=st[2]+' '+st[3]
    time=pd.to_datetime(t).strftime('%H:%M:%S')
    date_val.append(date)
    time_val.append(time)

In [ ]:
df_merged['Game Date'] = date_val
df_merged['Game Time'] = time_val

df_merged = df_merged.drop(columns=['Game Completed Date'])

In [ ]:
def convertToMinutes(col):
    for index, row in df_merged.iterrows():
        ts = df_merged[col][index]
        if ts=='-':
            pass
        else:
            ts = ts[:2]+':'+ts[4:6]
            ftr = [60,1]
            t = round(sum([a*b for a,b in zip(ftr, [int(i) for i in ts.split(":")])])/60,2)
            df_merged[col][index] = t

In [ ]:
convertToMinutes('Game Length')
convertToMinutes('Time to complete all tasks')

In [ ]:
df_merged['Game Length'] = df_merged['Game Length'].astype(float)
df_merged.head() 

<div>
    <h1><center style="background-color:#8394bf; color:white;">EDA</center></h1>
</div>

Used a new library called bar chart race for this viz🎨

[Documentation](https://github.com/dexplo/bar_chart_race)

In [ ]:
df_m= df_merged.groupby(['Game Date','User ID'], as_index=False)['Game Length'].mean()
df_m['User ID'] = df_m['User ID'].astype(str)
df_m['User ID'] = 'User '+ df_m['User ID']
df_plot = df_m.pivot(index='Game Date', columns='User ID', values='Game Length')

In [ ]:
bcr.bar_chart_race(df=df_plot,
                   orientation='h',
                   sort='desc',
                   n_bars=10,
                   label_bars=True,
                   steps_per_period=10,
                   period_length=800,
                   figsize=(4, 6),
                   cmap='dark12',
                   title='Average game length of users over time',
                   bar_label_size=7,
                   tick_label_size=7,
                   fig=None
                  )

![](https://media1.tenor.com/images/958277b6110090263b3b2190cc04ad9c/tenor.gif?itemid=19354055)

In [ ]:
win=[]
loss=[]
for i in range(0,len(li)):
    x = li[i]['Outcome'].value_counts()
    win.append(x[0])
    loss.append(x[1])

In [ ]:
dfo = pd.DataFrame(list(zip(win, loss)), columns = ['Win', 'Loss'])
dfo['User'] = (dfo.index+1).astype(str)
dfo['User'] = 'User '+ dfo['User']

In [ ]:
fig = go.Figure()
for col in dfo.columns[0:1]:
    fig.add_trace(go.Bar(x=dfo[col],
                         y=dfo['User'],
                         orientation='h',
                         name=col,
                         marker = dict(color = 'rgba(152, 190, 231, 1)',
                             line=dict(color=custom_colors[2],width=1)),
                         hovertemplate = "%{y}: %{x}"))
for col in dfo.columns[1:2]:
    fig.add_trace(go.Bar(x=-dfo[col].values,
                         y =dfo['User'],
                         orientation='h',
                         name= col,
                         customdata=dfo[col],
                         marker = dict(color = 'rgba(239, 77, 77, 0.5)',
                              line=dict(color=custom_colors[8],width=1)),
                         hovertemplate="%{y}: %{customdata}"))    
    
fig.add_layout_image(
    dict(
        source="https://i.imgur.com/Y27mTUq.png",
        xref="paper", yref="paper",
        x=0.7, y=1.05,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
    )
)
    
fig.add_layout_image(
    dict(
        source="https://i.imgur.com/z7Me2CG.png",
        xref="paper", yref="paper",
        x=0.3, y=1.05,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
    )
)
fig.add_layout_image(dict(
   source= "https://i.imgur.com/PomunFA.png",
   xref= "x",
   yref= "y",
   x= -20,
   y= 5,
   sizex= 50,
   sizey= 60,
   sizing= "contain",
   opacity= 0.8,
   visible = True,
   layer= "above")
)

fig.update_layout(barmode='relative', 
                  height=700, 
                  width=1000, 
                  yaxis_autorange='reversed',
                  bargap=0.01,
                  legend_orientation ='h',
                  legend_x=-0.05, legend_y=1.1,
                  title="Game Outcome",
                  title_x=0.39,
                  legend=dict(
                    yanchor="top",
                    y=0.99,
                    xanchor="left",
                    x=1),
                  template="plotly_white",paper_bgcolor='#F8F8F8'
                 )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [ ]:
df_crewmate = df_merged[df_merged['Team'] == 'Crewmate']
df_imposter = df_merged[df_merged['Team'] == 'Imposter']

value_counts = df_crewmate.groupby('User ID')['Ejected'].value_counts()
dfc = pd.DataFrame(value_counts.unstack(level=-1))
dfc = dfc.fillna(0)
dfc['User ID'] = dfc.reset_index().index+1

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=dfc['User ID'],
    y=dfc['Yes'],
    marker=dict(color=custom_colors[9], size=12),
    mode="markers+lines",
    connectgaps= True,
    name="Yes",
))

fig.add_trace(go.Scatter(
    x=dfc['User ID'],
    y=dfc['No'],
    marker=dict(color=custom_colors[4], size=12),
    mode="markers+lines",
    connectgaps= True,
    name="No",
))

fig.add_layout_image(dict(
   source= "https://i.imgur.com/BtZqOvN.png",
   xref= "x",
   yref= "y",
   x= 8,
   y= 80,
   sizex= 60,
   sizey= 60,
   sizing= "contain",
   opacity= 0.5,
   visible = True,
   layer= "below")
)

fig.update_layout(title="Crewmates and whether they were eliminated from the game",
                  yaxis_title="Count",
                 xaxis=dict(
                    title='User ID',
                    tickmode='linear'),
                 template="plotly_white",height=700,width=1000)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [ ]:
tc,ti = df_merged['Team'].value_counts()
cw,cl = df_crewmate['Outcome'].value_counts()
iw,il = df_imposter['Outcome'].value_counts()

In [ ]:
node_color = ["#4d194d", "#abc4ff", "#deaaff", "#c8e7ff","#E08DAC"]

link_color = ["#bde0fe","#cdb4db", "#ccdbfd", "#e2eafc", "#f3c4fb","#ffcbf2"]

fig = go.Figure(
            go.Sankey(
                    node = {
                    "label": ['Total Games','Crewmate', 'Imposter','Won','Lost'],
                    "color" : node_color
                    },
                    link = dict(
                        source = [0, 0, 1, 1, 2, 2],
                        target = [1, 2, 3, 4, 3, 4],
                        value = [tc, ti, cw, cl, iw,il],
                        color = link_color)
                    )
                )
fig.update_layout(title="Combined Game Stats",
                   paper_bgcolor='#FCFCFC',
                   height=700,
                   template="plotly_white")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [ ]:
fig = px.sunburst(df_merged, path=['User ID','Team', 'Outcome'],color_discrete_sequence=custom_colors) 
fig

🚧 Work in progress